# Choosing Appropriate Exposure Times and Dark-Frame Rules

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count

RE = RunEngine()
catalog = setup_data_saving(RE)

We will import some simulated devices from a Python script in the current directory, [simulated_hardware.py](./simulated_hardware.py). The objects present a programmatic interface that looks like real hardware, but their readings from come functions defined in another script, [generate_data.py](./generate_data.py), instead of from actual measurements.

## Manual Walk-through

First we'll take an exposure with the shutter open, then with it closed, and subtract the two, visualizing the results as we go.

In [ ]:
from simulated_hardware import detector, shutter

In [ ]:
RE(count([detector]))

In [ ]:
data = catalog[-1].primary.read()
data

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

In [ ]:
axes[0].imshow(data["detector_image"].mean("time"))

In [ ]:
fig

In [ ]:
shutter.set("closed").wait()

In [ ]:
RE(count([detector]))

In [ ]:
dark_data = catalog[-1].primary.read()
axes[1].imshow(dark_data["detector_image"].mean("time"))
fig

In [ ]:
subtracted = data["detector_image"].mean("time") - dark_data["detector_image"].mean("time")
axes[2].imshow(subtracted)
fig

Compute $I(q)$

In [ ]:
from utils import simple_integration

intensity = simple_integration(data["detector_image"].squeeze() - dark_data["detector_image"].squeeze())

These images were simulated based on a "ground truth" $I(q)$. Let's plot our computed $I(q)$ with the ground truth to check that they correspond closely.

In [ ]:
plt.figure()

In [ ]:
from simulated_hardware import intensities, x
plt.plot(x, intensities[0], "--", label="ground truth")
plt.plot(x[:len(intensity)], intensity, label="calculated")
plt.legend()
plt.gcf()

## Automatically capture dark frames

Use the utility [bluesky-darkframes](https://blueskyproject.io/bluesky-darkframes) to automatically capture darkframes as part of the data in every Run. This solves two problems for us:

1. We don't need to remember to take dark frames. They will be automatically taken for us.
2. Each Run is self-contained. Instead of having a "dark" Run and a "light" Run, every Run contains both light and dark together.

In [ ]:
from automatic_darkframes import setup_automatic_darkframes

dfp = setup_automatic_darkframes(RE)

In [ ]:
RE(count([detector]))

In [ ]:
data = catalog[-1].primary.read()
dark_data = catalog[-1].dark.read()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

In [ ]:
data["detector_image"].mean("time").plot(ax=axes[0])
dark_data["detector_image"].mean("time").plot(ax=axes[1])
(data["detector_image"].mean("time") - dark_data["detector_image"]).mean("time").plot(ax=axes[2])
fig.tight_layout()
fig